In [ ]:
# CONFIGURACIÓN DEL ENTORNO E IMPORTACIÓN DE LIBRERÍAS
# -------------------------------------------------------------
# Esta celda se encarga de importar todas las librerías necesarias para el proyecto.
# Es una buena práctica agrupar todas las importaciones al inicio del notebook.

# Para manipulación de datos
import pandas as pd
import numpy as np

# Para visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Para la adquisición de datos desde Kaggle
import kagglehub
import os # Para interactuar con el sistema operativo (manejo de archivos y rutas)
import zipfile # Para descomprimir archivos .zip

# Para preprocesamiento y modelado
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer # Para aplicar diferentes transformaciones a diferentes columnas
from sklearn.pipeline import Pipeline # Para encadenar pasos de preprocesamiento y modelado

# Modelos de Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb # Para XGBoost

# Métricas de evaluación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay, precision_recall_curve

# Para manejo de desequilibrio de clases (opcional, pero bueno tenerlo)
from imblearn.over_sampling import SMOTE

# Para interpretación de modelos (opcional)
# import shap # Descomentar si se va a usar SHAP

# Para ignorar advertencias (usar con precaución)
import warnings
warnings.filterwarnings('ignore')

# Configuraciones para mejorar la visualización de los gráficos
plt.style.use('seaborn-v0_8-whitegrid') # Estilo de los gráficos
plt.rcParams['figure.figsize'] = (12, 7) # Tamaño por defecto de las figuras
plt.rcParams['font.size'] = 12 # Tamaño por defecto de la fuente

print("Librerías importadas y configuración inicial lista.")

In [ ]:
# ADQUISICIÓN DE DATOS
# ---------------------------------------------------
import pandas as pd
import numpy as np
import os
import requests
from io import StringIO
import zipfile
from sklearn.datasets import load_breast_cancer

# Opción 1: Usar el dataset incorporado en scikit-learn
print("OPCIÓN 1: Cargando dataset de cáncer de mama desde scikit-learn...")
try:
    # Cargar el dataset integrado en scikit-learn
    breast_cancer = load_breast_cancer()
    
    # Crear DataFrame con los datos y características
    df = pd.DataFrame(data=breast_cancer.data, columns=breast_cancer.feature_names)
    
    # Añadir la columna objetivo (0=maligno, 1=benigno)
    df['diagnosis'] = breast_cancer.target
    
    # Convertir los valores numéricos a categorías de texto para mayor claridad
    df['diagnosis'] = df['diagnosis'].map({0: 'malignant', 1: 'benign'})
    
    # Guardar una copia del dataframe original
    df_original = df.copy()
    
    print("Dataset de scikit-learn cargado con éxito.")
    print(f"Dimensiones del dataset: {df.shape}")
    
    # Mostrar primeras filas del dataset
    print("\nPrimeras filas del dataset:")
    print(df.head())
    
except Exception as e:
    print(f"Error con scikit-learn: {e}")

# Opción 2: Descarga directa desde URL (GitHub u otra fuente confiable)
print("\n\nOPCIÓN 2: Intentando descarga directa desde GitHub...")
try:
    # URL de la fuente - usando una URL de ejemplo del dataset en GitHub
    url = "https://raw.githubusercontent.com/reihanenamdari/breast_cancer/master/data.csv"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # Leer el contenido CSV directamente desde la respuesta
        s = StringIO(response.text)
        df2 = pd.read_csv(s)
        df2_original = df2.copy()
        
        print("Dataset descargado exitosamente desde URL directa.")
        print(f"Dimensiones del dataset: {df2.shape}")
        
        # Mostrar primeras filas del dataset
        print("\nPrimeras filas del dataset:")
        print(df2.head())
        
        # Si el dataset de sklearn no se cargó, usar este
        if 'df' not in locals() or df.empty:
            df = df2
            df_original = df2_original
            print("\nUsando dataset de URL como dataset principal.")
    else:
        print(f"Error al descargar: Código de estado {response.status_code}")
        
except Exception as e:
    print(f"Error con descarga directa: {e}")

# Opción 3: Crear conjunto de datos sintético similar si todas las opciones anteriores fallan
if 'df' not in locals() or df.empty:
    print("\n\nOPCIÓN 3: Creando dataset sintético como último recurso...")
    try:
        # Generar un dataset sintético basado en la estructura conocida del dataset de cáncer de mama
        np.random.seed(42)  # Para reproducibilidad
        
        # Número de muestras
        n_samples = 569
        
        # Crear características con distribuciones similares al dataset real
        features = [
            'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 
            'smoothness_mean', 'compactness_mean', 'concavity_mean', 
            'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
            'radius_se', 'texture_se', 'perimeter_se', 'area_se', 
            'smoothness_se', 'compactness_se', 'concavity_se', 
            'concave points_se', 'symmetry_se', 'fractal_dimension_se',
            'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 
            'smoothness_worst', 'compactness_worst', 'concavity_worst', 
            'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst'
        ]
        
        # Diccionario para almacenar los datos generados
        data = {}
        
        # Generar datos para cada característica con rangos plausibles
        for feature in features:
            if 'radius' in feature:
                data[feature] = np.random.uniform(6, 28, n_samples)
            elif 'texture' in feature:
                data[feature] = np.random.uniform(9, 40, n_samples)
            elif 'perimeter' in feature:
                data[feature] = np.random.uniform(40, 190, n_samples)
            elif 'area' in feature:
                data[feature] = np.random.uniform(140, 2500, n_samples)
            elif 'smoothness' in feature:
                data[feature] = np.random.uniform(0.05, 0.16, n_samples)
            elif 'compactness' in feature:
                data[feature] = np.random.uniform(0.02, 0.35, n_samples)
            elif 'concavity' in feature:
                data[feature] = np.random.uniform(0, 0.5, n_samples)
            elif 'concave points' in feature:
                data[feature] = np.random.uniform(0, 0.2, n_samples)
            elif 'symmetry' in feature:
                data[feature] = np.random.uniform(0.1, 0.3, n_samples)
            elif 'fractal' in feature:
                data[feature] = np.random.uniform(0.05, 0.1, n_samples)
        
        # Crear el DataFrame con las características
        df = pd.DataFrame(data)
        
        # Generar etiquetas diagnósticas (aproximadamente 37% malignas, 63% benignas)
        diagnoses = np.random.choice(['malignant', 'benign'], size=n_samples, p=[0.37, 0.63])
        df['diagnosis'] = diagnoses
        
        # Guardar copia del original
        df_original = df.copy()
        
        print("Dataset sintético creado con éxito.")
        print(f"Dimensiones del dataset: {df.shape}")
        
        # Mostrar primeras filas del dataset
        print("\nPrimeras filas del dataset:")
        print(df.head())
        
    except Exception as e:
        print(f"Error al crear dataset sintético: {e}")

# Comprobar si tenemos un dataset válido
if 'df' in locals() and not df.empty:
    print("\n\nResumen del dataset final:")
    print(f"Número de filas: {df.shape[0]}")
    print(f"Número de columnas: {df.shape[1]}")
    print("\nInformación del dataset:")
    print(df.info())
    print("\nEstadísticas descriptivas:")
    print(df.describe())
    print("\nDistribución de diagnósticos:")
    print(df['diagnosis'].value_counts())
else:
    print("\n\nNo se pudo obtener un dataset válido. Por favor, verifica tu conexión a internet o considera descargar manualmente el dataset.")

In [ ]:
# INSPECCIÓN INICIAL DE DATOS
# -----------------------------------
# Con los datos ya cargados en nuestro DataFrame `df`,
# es momento de realizar una inspección básica para entender su estructura y contenido.
# Esto incluye verificar dimensiones, tipos de datos y visualizar algunas filas.

# `display` es preferible a `print` para DataFrames en entornos como Jupyter Notebooks,
# ya que ofrece una representación HTML más rica.

if 'df' in locals() and not df.empty:
    print("--- Iniciando Inspección General del Dataset ---")
    
    # Confirmamos las dimensiones: número de filas (observaciones) y columnas (variables).
    print(f"\nDimensiones del Dataset: {df.shape[0]} filas x {df.shape[1]} columnas.")

    # `df.info()` es muy útil: nos da un resumen conciso de cada columna,
    # su tipo de dato (Dtype) y el número de valores no nulos.
    # Esto ayuda a identificar rápidamente si hay tipos de datos inesperados o muchos faltantes.
    print("\nResumen de Información de las Columnas (df.info()):")
    df.info()

    # Visualizar las primeras filas nos da una idea tangible de los datos.
    print("\nPrimeras 5 filas del Dataset (df.head()):")
    display(df.head())

    # Visualizar las últimas filas también puede ser útil para detectar patrones o problemas al final del dataset.
    print("\nÚltimas 5 filas del Dataset (df.tail()):")
    display(df.tail())
    
    # Obtener una lista de todos los nombres de las columnas es útil para referencia futura
    # y para verificar que la carga se hizo correctamente.
    print("\nNombres de todas las Columnas:")
    print(list(df.columns))

else:
    # Mensaje de error si el DataFrame no está disponible.
    print("El DataFrame `df` parece estar vacío o no definido. No se puede realizar la inspección.")
    print("Por favor, asegúrese de que la Celda 2 (Adquisición de Datos) se haya ejecutado correctamente.")

In [ ]:
# ESTADÍSTICAS DESCRIPTIVAS Y VALORES NULOS/DUPLICADOS
# -----------------------------------------------------------
# Profundizamos en la inspección con estadísticas descriptivas más completas
# y una verificación exhaustiva de la presencia de valores ausentes (nulos)
# y filas duplicadas, aspectos críticos para la calidad de los datos.

if 'df' in locals() and not df.empty:
    # `df.describe(include='all')` nos da estadísticas para todas las columnas,
    # incluyendo las no numéricas (como nuestra columna 'diagnosis').
    # Para columnas numéricas: conteo, media, desviación estándar, mínimo, percentiles (25%, 50%, 75%) y máximo.
    # Para columnas categóricas/texto: conteo, número de valores únicos, el valor más frecuente (top) y su frecuencia (freq).
    print("\n--- Estadísticas Descriptivas Completas (df.describe(include='all')) ---")
    display(df.describe(include='all'))

    # Verificación detallada de valores nulos.
    print("\n--- Análisis de Valores Nulos por Columna ---")
    null_counts_per_column = df.isnull().sum()
    null_percentage_per_column = (df.isnull().sum() / len(df)) * 100
    
    # Creamos un DataFrame para presentar esta información de forma clara.
    null_analysis_df = pd.DataFrame({
        'Número de Nulos': null_counts_per_column,
        'Porcentaje de Nulos (%)': null_percentage_per_column
    })
    
    # Mostramos solo las columnas que efectivamente tienen valores nulos, ordenadas por porcentaje.
    missing_values_summary_df = null_analysis_df[null_analysis_df['Número de Nulos'] > 0].sort_values(
        by='Porcentaje de Nulos (%)', ascending=False
    )
    
    if not missing_values_summary_df.empty:
        print("Se han detectado valores nulos en las siguientes columnas:")
        display(missing_values_summary_df)
    elif null_counts_per_column.sum() == 0: # Doble verificación por si acaso
        print("¡Excelente! No se han encontrado valores nulos en ninguna columna del dataset.")
    else:
        # Este caso sería raro si el anterior se cumple, pero es una salvaguarda.
        print(f"No hay columnas con un conteo individual de nulos > 0, pero el total de nulos es: {null_counts_per_column.sum()}")


    # Verificación de filas completamente duplicadas.
    print("\n--- Análisis de Filas Duplicadas ---")
    number_of_duplicate_rows = df.duplicated().sum()
    print(f"Número total de filas duplicadas encontradas: {number_of_duplicate_rows}")
    
    if number_of_duplicate_rows == 0:
        print("No se han encontrado filas duplicadas en el dataset.")
    else:
        # Si hay duplicados, podríamos querer verlos (si no son demasiados).
        # display(df[df.duplicated(keep=False)].sort_values(by=list(df.columns))) # Muestra todas las ocurrencias de duplicados
        print("Se han detectado filas duplicadas. Se revisará su manejo en la etapa de limpieza de datos.")

else:
    print("El DataFrame `df` está vacío o no definido. Imposible realizar análisis de calidad de datos.")
    print("Por favor, verifique la ejecución de las celdas anteriores.")

In [ ]:
# LIMPIEZA DE DATOS
# -------------------------
# Basándonos en los hallazgos de la inspección inicial (Celdas 3 y 4),
# procedemos con las tareas de limpieza necesarias. Esto puede incluir
# corrección de nombres de columnas, manejo de duplicados, y ajuste de tipos de datos.

if 'df' in locals() and not df.empty:
    print("--- Iniciando Proceso de Limpieza de Datos ---")

    # 1. Corrección de Nombres de Columnas:
    #    Aseguramos que los nombres sean consistentes y fáciles de usar (sin espacios extra, usando guiones bajos).
    print("\nNombres de columnas antes de la corrección:", df.columns.tolist())
    original_column_names = df.columns.tolist() # Guardamos por si hay cambios
    
    df.columns = df.columns.str.strip() # Elimina espacios en blanco al inicio y al final.
    df.columns = df.columns.str.replace(' ', '_') # Reemplaza espacios intermedios por guiones bajos.
                                                 # También convierte a minúsculas para consistencia (opcional pero común).
    df.columns = df.columns.str.lower()
    
    if df.columns.tolist() != original_column_names:
        print("Nombres de columnas después de la corrección:", df.columns.tolist())
    else:
        print("No fue necesario modificar los nombres de las columnas (ya estaban en formato adecuado).")
    
    # 2. Manejo de Filas Duplicadas:
    #    Si se detectaron filas duplicadas en la celda anterior, aquí las eliminamos.
    #    Usualmente, se conserva la primera aparición ('keep=first').
    num_duplicates_before = df.duplicated().sum()
    if num_duplicates_before > 0:
        print(f"\nSe encontraron {num_duplicates_before} filas duplicadas. Procediendo a eliminarlas...")
        df.drop_duplicates(inplace=True, keep='first')
        print(f"Filas duplicadas eliminadas. Nuevas dimensiones del DataFrame: {df.shape}")
        # Es buena práctica resetear el índice después de eliminar filas.
        df.reset_index(drop=True, inplace=True)
        print("Índice del DataFrame reseteado.")
    else:
        print("\nNo se encontraron filas duplicadas para eliminar.")

    # 3. Transformación de la Variable Objetivo (si es necesario y no se hizo antes):
    #    En la Celda 2, mapeamos 'diagnosis' de 0/1 a 'malignant'/'benign'.
    #    Para los modelos, necesitaremos que sea numérica (0/1). Esta transformación
    #    se hará más adelante (Celda 11) de forma más específica para la preparación de X e y.
    #    Este bloque es un recordatorio de que si tuviéramos otras columnas objetivo textuales,
    #    aquí sería un buen lugar para mapearlas a números.
    #    Por ejemplo, si 'diagnosis' aún no fuera 'category':
    if 'diagnosis' in df.columns and df['diagnosis'].dtype == 'object':
         print("\nLa columna 'diagnosis' es de tipo 'object'. Convirtiendo a 'category' para optimización.")
         df['diagnosis'] = df['diagnosis'].astype('category')
    elif 'diagnosis' in df.columns and pd.api.types.is_categorical_dtype(df['diagnosis']):
         print("\nLa columna 'diagnosis' ya es de tipo 'category'.")
    else:
         print("\nLa columna 'diagnosis' no se encontró o no es 'object' ni 'category'. Revisar.")


    # 4. Asegurar Tipos de Datos Categóricos para otras variables (si las hubiera):
    #    Si alguna otra columna de texto debiera ser tratada como una categoría,
    #    la convertimos explícitamente. Esto puede optimizar el uso de memoria y
    #    es requerido por algunas funciones de pandas o scikit-learn.
    print("\nAsegurando que otras columnas textuales con baja cardinalidad sean de tipo 'category'...")
    changed_types_count = 0
    for col_name in df.select_dtypes(include=['object']).columns:
        # Excluimos la columna 'diagnosis' ya que la manejamos/manejaremos específicamente.
        if col_name == 'diagnosis':
            continue
        
        # Consideramos una columna como categórica si tiene un número de valores únicos
        # relativamente bajo en comparación con el total de filas.
        # Y más de 1 valor único (si solo tiene 1, no aporta mucha información como categoría).
        if 1 < df[col_name].nunique() < len(df) * 0.1: # Umbral del 10% de valores únicos
            print(f"Convirtiendo la columna '{col_name}' (con {df[col_name].nunique()} valores únicos) a tipo 'category'.")
            df[col_name] = df[col_name].astype('category')
            changed_types_count += 1
    if changed_types_count == 0 and not any(col == 'diagnosis' for col in df.select_dtypes(include=['object']).columns):
        print("No se identificaron otras columnas 'object' para convertir a 'category' (o ya lo son).")


    print("\n--- Proceso de Limpieza de Datos Completado ---")
    print("\nInformación actualizada del DataFrame (df.info()):")
    df.info()
    
else:
    print("El DataFrame `df` está vacío o no definido. No se puede proceder con la limpieza.")

In [ ]:
# ANÁLISIS EXPLORATORIO DE DATOS (EDA) - VARIABLE OBJETIVO 
# ------------------------------------------------------------------------
# Analizamos la distribución de la variable objetivo (diagnosis en nuestro dataset actual)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Primero verificamos qué columnas están disponibles en nuestro dataset
print("Columnas disponibles en el dataset:")
print(df.columns.tolist())

# Identificar la variable objetivo (probablemente 'diagnosis' en vez de 'Status')
target_column = None

# Buscar la columna objetivo más probable
if not df.empty:
    potential_targets = ['diagnosis', 'target', 'label', 'class', 'Status']
    for col in potential_targets:
        if col in df.columns:
            target_column = col
            break
    
    # Si no encontramos ninguna de las columnas típicas, intentamos detectar automáticamente
    if target_column is None:
        # Buscar columnas con pocas categorías (típicamente 2-5)
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
        for col in categorical_cols:
            if df[col].nunique() <= 5:  # Asumimos que una columna categórica con pocas categorías podría ser el objetivo
                target_column = col
                print(f"Se detectó automáticamente '{col}' como posible variable objetivo.")
                break

# Si aún no tenemos variable objetivo y tenemos pocas columnas, mostramos los valores únicos de cada una
if target_column is None and df.shape[1] < 15:
    print("\nNo se identificó automáticamente una variable objetivo. Mostrando valores únicos de cada columna:")
    for col in df.columns:
        unique_values = df[col].nunique()
        if unique_values < 10:  # Solo mostramos columnas con pocos valores únicos
            print(f"Columna '{col}': {unique_values} valores únicos - {df[col].unique()}")

# Proceder con el análisis si encontramos una variable objetivo
if target_column is not None:
    print(f"\n--- Análisis de la Variable Objetivo ('{target_column}') ---")
    
    target_counts = df[target_column].value_counts()
    target_percentage = df[target_column].value_counts(normalize=True) * 100
    
    print("\nConteo por categoría:")
    print(target_counts)
    print("\nPorcentaje por categoría:")
    print(target_percentage)
    
    # Crear un mapa de colores para las categorías
    if len(target_counts) == 2:
        color_palette = ['#3498db', '#e74c3c']  # Azul y rojo para caso binario
    else:
        color_palette = sns.color_palette("viridis", len(target_counts))
    
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(x=target_column, data=df, palette=color_palette)
    plt.title(f'Distribución de la Variable Objetivo ({target_column})', fontsize=16)
    plt.xlabel(target_column, fontsize=14)
    plt.ylabel('Número de Muestras', fontsize=14)
    
    # Generar etiquetas descriptivas para el eje x si son valores numéricos
    if df[target_column].dtype in [np.int64, np.int32, np.float64]:
        category_labels = [f"{val}" for val in sorted(df[target_column].unique())]
        plt.xticks(ticks=range(len(category_labels)), labels=category_labels)
    
    # Añadir anotaciones de porcentaje
    for i, count_val in enumerate(target_counts):
        percentage_val = target_percentage[target_counts.index[i]]
        plt.text(i, count_val + (max(target_counts) * 0.05), 
                 f'{percentage_val:.1f}% ({count_val})', 
                 ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    # Comprobar si hay desequilibrio de clases
    if len(target_percentage) == 2:  # Solo verificamos desequilibrio para problemas binarios
        imbalance_ratio = max(target_percentage) / min(target_percentage)
        if imbalance_ratio > 1.5:  # Umbral arbitrario para desequilibrio
            print(f"\nObservación: El dataset presenta un desequilibrio de clases (ratio {imbalance_ratio:.2f}:1).")
            print("Esto deberá tenerse en cuenta durante el modelado (e.g., usando métricas apropiadas, stratify en train_test_split, o técnicas de re-muestreo).")
    
    # Mostrar distribución en formato pie chart para mejor visualización
    plt.figure(figsize=(10, 6))
    explode = [0.05] * len(target_counts)  # Explotar ligeramente todas las porciones
    plt.pie(target_counts, labels=[f"{idx} ({val})" for idx, val in target_counts.items()], 
            autopct='%1.1f%%', startangle=90, explode=explode, 
            colors=color_palette, shadow=True)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.title(f'Distribución de {target_column} (Gráfico de Torta)', fontsize=16)
    plt.tight_layout()
    plt.show()
    
else:
    print("\nNo se pudo identificar una columna adecuada como variable objetivo.")
    print("Por favor, especifica manualmente la columna objetivo a analizar.")
    
    # Mostrar un resumen de todas las columnas para ayudar al usuario a identificar la variable objetivo
    print("\nResumen de todas las columnas disponibles:")
    for col in df.columns:
        dtype = df[col].dtype
        n_unique = df[col].nunique()
        if n_unique < 20:  # Solo mostrar valores únicos si son pocos
            print(f"- {col}: {dtype}, {n_unique} valores únicos: {df[col].unique()}")
        else:
            print(f"- {col}: {dtype}, {n_unique} valores únicos")

In [ ]:
# EDA - ANÁLISIS UNIVARIADO DE CARACTERÍSTICAS NUMÉRICAS
# ---------------------------------------------------------------
# Después de analizar la variable objetivo, exploramos cada característica numérica de forma individual.
# El objetivo es entender la distribución de cada una: su forma, tendencia central, dispersión,
# y la presencia de posibles valores atípicos (outliers).
# Usaremos histogramas y diagramas de caja (boxplots) para esta tarea.

# matplotlib.pyplot y seaborn ya deberían estar importados.
# numpy también.

if 'df' in locals() and not df.empty:
    print("--- EDA: Análisis Univariado de Características Numéricas ---")
    
    # Seleccionamos las columnas que son de tipo numérico.
    # Excluimos cualquier columna que sea la variable objetivo si esta fuera numérica (aunque 'diagnosis' es categórica).
    # También es buena práctica excluir columnas que parezcan IDs si son numéricas.
    
    numeric_cols_for_eda = df.select_dtypes(include=np.number).columns.tolist()
    
    # Si 'target_col_eda' (definida en Celda 6) es una columna numérica, la removemos de esta lista.
    # Esto es poco probable en nuestro caso ya que 'diagnosis' es el objetivo y es categórico.
    if 'target_col_eda' in locals() and target_col_eda in numeric_cols_for_eda:
        print(f"Excluyendo la columna objetivo '{target_col_eda}' del análisis de características numéricas.")
        numeric_cols_for_eda.remove(target_col_eda)
        
    # Identificar y excluir posibles columnas de ID que puedan ser numéricas.
    # Un ID suele tener tantos valores únicos como filas, o contener 'id' en su nombre.
    potential_id_cols = [
        col for col in numeric_cols_for_eda 
        if 'id' in col.lower() or df[col].nunique() >= len(df) * 0.95 # Si casi todos los valores son únicos
    ]
    if potential_id_cols:
        print(f"Excluyendo posibles columnas ID del análisis numérico: {potential_id_cols}")
        for id_col in potential_id_cols:
            if id_col in numeric_cols_for_eda: # Comprobar si aún está en la lista
                numeric_cols_for_eda.remove(id_col)

    if numeric_cols_for_eda:
        print(f"Se analizarán {len(numeric_cols_for_eda)} características numéricas.")
        
        # Definimos cómo organizar los múltiples gráficos.
        num_plots_per_row = 3
        num_rows_for_plots = int(np.ceil(len(numeric_cols_for_eda) / num_plots_per_row))

        # 1. Histogramas para visualizar la distribución (forma, sesgo, multimodalidad).
        #    KDE (Kernel Density Estimate) superpuesto ayuda a ver la forma de la distribución.
        plt.figure(figsize=(num_plots_per_row * 5, num_rows_for_plots * 4)) # Ajustar tamaño dinámicamente
        plt.suptitle("Distribución de Características Numéricas (Histogramas + KDE)", fontsize=18, weight='bold', y=1.03)
        for i, col_name in enumerate(numeric_cols_for_eda):
            plt.subplot(num_rows_for_plots, num_plots_per_row, i + 1)
            sns.histplot(df[col_name], kde=True, bins=30, color='skyblue', edgecolor='black', line_kws={'linewidth': 2})
            plt.title(f'{col_name.replace("_", " ").capitalize()}', fontsize=14)
            plt.xlabel(col_name.replace("_", " ").capitalize(), fontsize=12)
            plt.ylabel('Frecuencia', fontsize=12)
            plt.xticks(fontsize=10)
            plt.yticks(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout(rect=[0, 0, 1, 0.98]) # Ajuste para el supertítulo
        plt.show()

        # 2. Diagramas de Caja (Boxplots) para identificar la mediana, cuartiles y valores atípicos.
        plt.figure(figsize=(num_plots_per_row * 5, num_rows_for_plots * 4))
        plt.suptitle("Dispersión de Características Numéricas (Diagramas de Caja)", fontsize=18, weight='bold', y=1.03)
        for i, col_name in enumerate(numeric_cols_for_eda):
            plt.subplot(num_rows_for_plots, num_plots_per_row, i + 1)
            sns.boxplot(y=df[col_name], color='lightcoral', width=0.5)
            plt.title(f'{col_name.replace("_", " ").capitalize()}', fontsize=14)
            plt.ylabel(col_name.replace("_", " ").capitalize(), fontsize=12)
            plt.xticks(fontsize=10) # Aunque no hay etiquetas X, para consistencia del tamaño de fuente
            plt.yticks(fontsize=10)
            plt.grid(True, linestyle='--', alpha=0.7, axis='y') # Rejilla solo en el eje Y
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()
    else:
        print("No se encontraron características numéricas (distintas de la objetivo o IDs) para analizar.")
else:
    print("El DataFrame `df` está vacío o no definido. No se puede realizar el análisis univariado numérico.")

In [ ]:
# ANÁLISIS UNIVARIADO DE CARACTERÍSTICAS CATEGÓRICAS
# -----------------------------------------------------------------
# Después de las numéricas, analizamos las características categóricas.
# Estas son variables que representan grupos o categorías, como 'tipo A', 'tipo B', etc.
# En nuestro dataset de cáncer de mama, la principal variable categórica es 'diagnosis'.
# Si tuviéramos otras (por ejemplo, 'estadio del tumor' si fuera textual), las analizaríamos aquí.
# El objetivo es ver la frecuencia de cada categoría.

# matplotlib.pyplot y seaborn ya deberían estar importados.

if 'df' in locals() and not df.empty:
    print("--- EDA: Análisis Univariado de Características Categóricas ---")
    
    # Seleccionamos columnas de tipo 'category' u 'object' (texto).
    # La variable 'target_col_eda' fue identificada en la Celda 6 (debería ser 'diagnosis').
    # Aquí, estamos interesados en la distribución de 'diagnosis' misma como categórica,
    # y cualquier OTRA característica predictora que sea categórica.
    
    categorical_cols_for_eda = df.select_dtypes(include=['category', 'object']).columns.tolist()
    
    # Si no hay características categóricas, lo indicamos.
    if not categorical_cols_for_eda:
        print("No se encontraron características categóricas para analizar en este paso.")
    else:
        print(f"Se analizarán {len(categorical_cols_for_eda)} características categóricas: {categorical_cols_for_eda}")
        
        # Definimos cómo organizar los gráficos.
        num_plots_per_row_cat = 2  # Ajustar si hay muchas o pocas
        if len(categorical_cols_for_eda) == 1:
            num_plots_per_row_cat = 1 # Si solo hay una, que ocupe más espacio
            
        num_rows_for_plots_cat = int(np.ceil(len(categorical_cols_for_eda) / num_plots_per_row_cat))

        if num_rows_for_plots_cat > 0: # Solo crear figura si hay algo que plotear
            plt.figure(figsize=(num_plots_per_row_cat * 7, num_rows_for_plots_cat * 5)) # Tamaño dinámico
            plt.suptitle("Distribución de Características Categóricas", fontsize=18, weight='bold', y=1.03 if len(categorical_cols_for_eda) > 1 else 1.05)

            for i, col_name in enumerate(categorical_cols_for_eda):
                plt.subplot(num_rows_for_plots_cat, num_plots_per_row_cat, i + 1)
                
                # Ordenamos las barras por frecuencia para una mejor visualización.
                value_order = df[col_name].value_counts().index
                
                # Usamos countplot. Si las etiquetas de categoría son largas, 'orient="h"' (o y=col_name) es mejor.
                ax_cat_countplot = sns.countplot(
                    data=df, 
                    y=col_name, # Barras horizontales, bueno para etiquetas de categoría
                    order=value_order, 
                    palette='viridis_r' # Paleta de colores
                )
                
                plt.title(f'{col_name.replace("_", " ").capitalize()}', fontsize=14)
                plt.xlabel('Conteo de Observaciones', fontsize=12)
                plt.ylabel(col_name.replace("_", " ").capitalize(), fontsize=12)
                plt.xticks(fontsize=10)
                plt.yticks(fontsize=10)
                plt.grid(True, linestyle='--', alpha=0.7, axis='x') # Rejilla en el eje X para barras horizontales

                # Añadir conteos al final de cada barra.
                for p_patch in ax_cat_countplot.patches:
                    width = p_patch.get_width()
                    ax_cat_countplot.text(
                        width + (ax_cat_countplot.get_xlim()[1] * 0.01), # Pequeño offset del final de la barra
                        p_patch.get_y() + p_patch.get_height() / 2.,
                        f'{int(width)}', # El conteo
                        va='center', 
                        ha='left', # Alineado a la izquierda del texto
                        fontsize=10, 
                        color='black'
                    )
            
            plt.tight_layout(rect=[0, 0, 1, 0.97]) # Ajuste para el supertítulo
            plt.show()
        elif categorical_cols_for_eda: # Hay columnas pero no se generaron filas para plots (raro)
             print(f"Se identificaron columnas categóricas {categorical_cols_for_eda} pero no se generaron gráficos. Revisar lógica.")

else:
    print("El DataFrame `df` está vacío o no definido. No se puede realizar el análisis univariado categórico.")

In [ ]:
# ANÁLISIS BIVARIADO (CARACTERÍSTICAS VS. VARIABLE OBJETIVO)
# ------------------------------------------------------------------------
# Esta es una etapa crucial del EDA. Aquí exploramos cómo cada característica predictora
# se relaciona con nuestra variable objetivo ('diagnosis').
# Buscamos identificar qué características parecen tener una influencia o una
# distribución diferente para los casos benignos versus los malignos.
# Esto nos da pistas tempranas sobre qué variables podrían ser más importantes para nuestros modelos.

# matplotlib.pyplot, seaborn, pandas, numpy ya deberían estar disponibles.
# from sklearn.preprocessing import LabelEncoder # Para codificar la objetivo para correlaciones.

if 'df' in locals() and not df.empty and 'target_col_eda' in locals() and target_col_eda:
    # 'target_col_eda' fue definida en la Celda 6 (debería ser 'diagnosis').
    # 'numeric_cols_for_eda' fue definida en la Celda 7.
    # 'categorical_cols_for_eda' fue definida en la Celda 8.

    print(f"--- EDA: Análisis Bivariado (Características Predictoras vs. '{target_col_eda}') ---")
    
    # Usaremos una copia para cualquier transformación temporal (como codificar 'diagnosis' para correlaciones).
    df_bivariate_analysis = df.copy()

    # Colores consistentes para las clases de la variable objetivo.
    palette_for_target = {'benign': '#3498db', 'malignant': '#e74c3c'} if target_col_eda == 'diagnosis' and set(df[target_col_eda].unique()) == {'benign', 'malignant'} else sns.color_palette("husl", df[target_col_eda].nunique())


    # 1. Características Numéricas Predictoras vs. Variable Objetivo.
    #    Visualizamos con boxplots y violinplots para comparar distribuciones.
    if 'numeric_cols_for_eda' in locals() and numeric_cols_for_eda:
        print(f"\n--- Relación: Características Numéricas vs. '{target_col_eda}' ---")
        
        # Limitamos el número de gráficos para no saturar; podríamos seleccionar las más prometedoras
        # o las primeras N. Aquí tomamos las primeras 9-12 para el ejemplo.
        features_to_plot_num_vs_target = numeric_cols_for_eda[:min(12, len(numeric_cols_for_eda))]
        
        num_plot_cols_biv = 3
        num_plot_rows_biv = int(np.ceil(len(features_to_plot_num_vs_target) / num_plot_cols_biv))

        if num_plot_rows_biv > 0:
            # Boxplots: Comparan la distribución (mediana, cuartiles, outliers) de cada numérica para cada clase del target.
            plt.figure(figsize=(num_plot_cols_biv * 6, num_plot_rows_biv * 5))
            plt.suptitle(f"Características Numéricas vs. {target_col_eda.replace('_',' ').capitalize()} (Diagramas de Caja)", fontsize=18, weight='bold', y=1.03)
            for i, col in enumerate(features_to_plot_num_vs_target):
                plt.subplot(num_plot_rows_biv, num_plot_cols_biv, i + 1)
                sns.boxplot(x=target_col_eda, y=col, data=df_bivariate_analysis, palette=palette_for_target, width=0.6)
                plt.title(f'{col.replace("_"," ").capitalize()}', fontsize=14)
                plt.xlabel(target_col_eda.replace("_"," ").capitalize(), fontsize=12)
                plt.ylabel(col.replace("_"," ").capitalize(), fontsize=12)
                plt.xticks(fontsize=11); plt.yticks(fontsize=11)
                plt.grid(True, linestyle='--', alpha=0.6, axis='y')
            plt.tight_layout(rect=[0, 0, 1, 0.98])
            plt.show()

            # Violinplots: Similar a boxplots pero también muestran la forma de la distribución (como un KDE).
            plt.figure(figsize=(num_plot_cols_biv * 6, num_plot_rows_biv * 5))
            plt.suptitle(f"Características Numéricas vs. {target_col_eda.replace('_',' ').capitalize()} (Diagramas de Violín)", fontsize=18, weight='bold', y=1.03)
            for i, col in enumerate(features_to_plot_num_vs_target):
                plt.subplot(num_plot_rows_biv, num_plot_cols_biv, i + 1)
                sns.violinplot(x=target_col_eda, y=col, data=df_bivariate_analysis, palette=palette_for_target, inner='quartile', cut=0)
                plt.title(f'{col.replace("_"," ").capitalize()}', fontsize=14)
                plt.xlabel(target_col_eda.replace("_"," ").capitalize(), fontsize=12)
                plt.ylabel(col.replace("_"," ").capitalize(), fontsize=12)
                plt.xticks(fontsize=11); plt.yticks(fontsize=11)
                plt.grid(True, linestyle='--', alpha=0.6, axis='y')
            plt.tight_layout(rect=[0, 0, 1, 0.98])
            plt.show()
        else:
            print("No hay suficientes características numéricas seleccionadas para graficar contra el objetivo.")
    else:
        print("No se encontraron características numéricas predictoras para el análisis bivariado.")

    # 2. Características Categóricas Predictoras (si las hay) vs. Variable Objetivo.
    #    Usamos tablas de contingencia y gráficos de barras apiladas.
    #    En nuestro caso, 'categorical_cols_for_eda' de la Celda 8 probablemente solo contenía 'diagnosis'.
    #    Aquí buscamos OTRAS predictoras categóricas.
    other_cat_predictors = [col for col in df_bivariate_analysis.select_dtypes(include=['category', 'object']).columns if col != target_col_eda]
    
    if other_cat_predictors:
        print(f"\n--- Relación: Características Categóricas Predictoras vs. '{target_col_eda}' ---")
        num_plot_cols_cat_biv = 2
        num_plot_rows_cat_biv = int(np.ceil(len(other_cat_predictors) / num_plot_cols_cat_biv))

        if num_plot_rows_cat_biv > 0:
            plt.figure(figsize=(num_plot_cols_cat_biv * 8, num_plot_rows_cat_biv * 6))
            plt.suptitle(f"Predictoras Categóricas vs. {target_col_eda.replace('_',' ').capitalize()} (Barras Apiladas %)", fontsize=18, weight='bold', y=1.03)
            for i, col in enumerate(other_cat_predictors):
                ax_sub = plt.subplot(num_plot_rows_cat_biv, num_plot_cols_cat_biv, i + 1)
                # Creamos una tabla de contingencia normalizada por índice (porcentaje por cada categoría de 'col')
                contingency_table_norm = pd.crosstab(df_bivariate_analysis[col], df_bivariate_analysis[target_col_eda], normalize='index') * 100
                contingency_table_norm.plot(kind='barh', stacked=True, ax=ax_sub, colormap='coolwarm_r', width=0.8)
                plt.title(f'{col.replace("_"," ").capitalize()}', fontsize=14)
                plt.xlabel(f'Porcentaje de {target_col_eda.replace("_"," ").capitalize()}', fontsize=12)
                plt.ylabel(col.replace("_"," ").capitalize(), fontsize=12)
                plt.xticks(fontsize=11); plt.yticks(fontsize=11)
                plt.legend(title=target_col_eda.replace("_"," ").capitalize(), bbox_to_anchor=(1.02, 1), loc='upper left')
                # Añadir etiquetas de porcentaje dentro de las barras
                for n, (idx, row) in enumerate(contingency_table_norm.iterrows()):
                    cumulative_width = 0
                    for m, val in enumerate(row):
                        width = val
                        if width > 5: # Solo mostrar si el segmento es suficientemente grande
                            plt.text(cumulative_width + width/2, n, f"{width:.1f}%", 
                                     va='center', ha='center', fontsize=9, color='white' if width > 40 else 'black')
                        cumulative_width += width
            plt.tight_layout(rect=[0, 0, 1, 0.97])
            plt.show()
    else:
        print(f"\nNo se encontraron OTRAS características categóricas predictoras para comparar con '{target_col_eda}'.")

    # 3. Matriz de Correlación (entre numéricas y la objetivo codificada).
    #    Reutilizamos 'numeric_cols_for_eda'.
    if 'numeric_cols_for_eda' in locals() and numeric_cols_for_eda:
        print("\n--- Correlación entre Características Numéricas y la Variable Objetivo (Codificada) ---")
        df_for_correlation = df_bivariate_analysis[numeric_cols_for_eda].copy()
        
        # Necesitamos la variable objetivo en formato numérico para la correlación.
        # Usamos LabelEncoder: asignará 0 a una clase y 1 a la otra (alfabéticamente si no se especifica).
        # Si 'benign' es 0 y 'malignant' es 1, una correlación positiva con 'diagnosis_encoded'
        # significa que al aumentar la característica, aumenta la probabilidad de ser 'malignant'.
        le = LabelEncoder()
        df_for_correlation[target_col_eda + "_encoded"] = le.fit_transform(df_bivariate_analysis[target_col_eda])
        
        print(f"Clases codificadas para '{target_col_eda}': {list(zip(le.classes_, le.transform(le.classes_)))}")

        correlation_matrix_with_target = df_for_correlation.corr()
        
        # Extraemos solo la columna de correlaciones con la variable objetivo.
        target_correlations = correlation_matrix_with_target[target_col_eda + "_encoded"].drop(target_col_eda + "_encoded")
        target_correlations_sorted = target_correlations.sort_values(ascending=False)
        
        print("\nCorrelaciones de Pearson más significativas con la variable objetivo codificada:")
        print("Top 10 Positivas (más asociadas con la clase '1' - e.g., 'malignant'):")
        print(target_correlations_sorted.head(10))
        print("\nTop 10 Negativas (más asociadas con la clase '0' - e.g., 'benign'):")
        print(target_correlations_sorted.abs().sort_values(ascending=False).tail(10) if target_correlations_sorted.min() < 0 else "No hay correlaciones negativas significativas.")


        # Visualización de estas correlaciones.
        plt.figure(figsize=(10, 8))
        # Colores diferentes para correlaciones positivas y negativas.
        colors = ['crimson' if c < 0 else 'royalblue' for c in target_correlations_sorted.values]
        target_correlations_sorted.plot(kind='bar', color=colors)
        plt.title(f'Correlación de Características con {target_col_eda.replace("_"," ").capitalize()} (Codificada)', fontsize=16, weight='bold')
        plt.ylabel('Coeficiente de Correlación de Pearson', fontsize=12)
        plt.xlabel('Características', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.axhline(0, color='black', linewidth=0.8) # Línea en cero
        plt.tight_layout()
        plt.show()
    else:
        print("No se pudieron calcular correlaciones con el objetivo (faltan características numéricas o el objetivo).")

else:
    print("El DataFrame `df` está vacío o la variable objetivo `target_col_eda` no fue definida. No se puede realizar análisis bivariado.")

In [ ]:
# EDA - MATRIZ DE CORRELACIÓN (ENTRE CARACTERÍSTICAS NUMÉRICAS PREDICTORAS)
# ---------------------------------------------------------------------------------
# En la celda anterior, vimos la correlación de cada característica numérica con la variable objetivo.
# Ahora, vamos a analizar la correlación ENTRE las propias características predictoras numéricas.
# El objetivo es identificar si existen pares o grupos de características que estén fuertemente
# correlacionadas entre sí (lo que se conoce como multicolinealidad).
# Esto es importante porque algunos modelos pueden ser sensibles a la multicolinealidad,
# y también nos indica si algunas variables podrían estar proporcionando información redundante.

# matplotlib.pyplot, seaborn, pandas, numpy ya deberían estar disponibles.

if 'df' in locals() and not df.empty:
    # 'numeric_cols_for_eda' fue definida en la Celda 7 y ya excluye IDs y la objetivo si fuera numérica.
    if 'numeric_cols_for_eda' in locals() and numeric_cols_for_eda:
        
        if len(numeric_cols_for_eda) > 1:
            print("--- EDA: Matriz de Correlación entre Características Numéricas Predictoras ---")

            # Calculamos la matriz de correlación de Pearson para las características seleccionadas.
            correlation_matrix_predictors_only = df[numeric_cols_for_eda].corr()
            
            # Visualizamos esta matriz usando un mapa de calor (heatmap).
            # Un heatmap es ideal para representar matrices de correlación,
            # usando colores para indicar la fuerza y dirección de la correlación.
            plt.figure(figsize=(20, 18)) # Hacemos esta figura bastante grande para que se lean las etiquetas
            sns.heatmap(
                correlation_matrix_predictors_only,
                annot=True,        # Muestra los valores numéricos de correlación en cada celda.
                cmap='coolwarm',   # Paleta de colores: 'coolwarm' va de azul (negativo) a rojo (positivo), pasando por blanco (cero).
                fmt=".2f",         # Formato de los números (2 decimales).
                linewidths=.5,     # Líneas delgadas para separar las celdas.
                cbar_kws={"shrink": .8} # Ajustar el tamaño de la barra de color.
            )
            plt.title('Matriz de Correlación entre Características Numéricas Predictoras', fontsize=20, weight='bold', pad=20)
            plt.xticks(rotation=45, ha='right', fontsize=10) # Rotar etiquetas del eje X para mejor lectura
            plt.yticks(fontsize=10)
            plt.tight_layout()
            plt.show()
            
            print("\nInterpretación de la Matriz de Correlación entre Predictoras:")
            print(" - Valores cercanos a +1 indican una fuerte correlación positiva (si una sube, la otra tiende a subir).")
            print(" - Valores cercanos a -1 indican una fuerte correlación negativa (si una sube, la otra tiende a bajar).")
            print(" - Valores cercanos a 0 indican una correlación lineal débil o nula.")
            print(" - Una alta correlación entre dos (o más) variables predictoras (multicolinealidad) puede ser problemática:")
            print("   * Para la interpretabilidad de los coeficientes en modelos lineales.")
            print("   * Algunos algoritmos pueden volverse inestables o menos eficientes.")
            print("   * Podría indicar que algunas características son redundantes.")
            print(" - No siempre es necesario eliminar características altamente correlacionadas, pero es fundamental ser consciente de ello.")

        else:
            print("No hay suficientes características numéricas predictoras (se necesita más de una) para generar una matriz de correlación.")
    else:
        print("La lista de características numéricas predictoras ('numeric_cols_for_eda') no está definida. Verifique la Celda 7.")
else:
    print("El DataFrame `df` está vacío o no definido. No se puede generar la matriz de correlación.")

In [ ]:
# PREPROCESAMIENTO - CODIFICACIÓN DE LA VARIABLE OBJETIVO
# -----------------------------------------------------------------
# Los algoritmos de Machine Learning de scikit-learn requieren que tanto las características
# predictoras (X) como la variable objetivo (y) sean numéricas.
# Nuestra variable objetivo 'diagnosis' actualmente contiene etiquetas textuales ('malignant', 'benign').
# En este paso, la convertiremos a un formato numérico (generalmente 0 y 1).

# pandas ya debería estar importado.

if 'df' in locals() and not df.empty:
    print("--- Preprocesamiento: Codificación de la Variable Objetivo 'diagnosis' ---")

    # Hacemos una copia del DataFrame para este paso específico de preprocesamiento.
    # Si bien podríamos modificar `df` directamente, crear `df_for_modeling`
    # nos permite mantener `df` con 'diagnosis' textual por si lo necesitamos para EDA posterior.
    # Sin embargo, para simplificar el flujo y dado que las celdas siguientes usarán `df`,
    # actualizaremos `df` directamente después de la codificación.
    
    # df_for_modeling = df.copy() # Opción si quisiéramos mantener df intacto.

    if 'diagnosis' in df.columns and (df['diagnosis'].dtype == 'object' or pd.api.types.is_categorical_dtype(df['diagnosis'])):
        
        print(f"\nValores únicos en 'diagnosis' antes de la codificación: {df['diagnosis'].unique().tolist()}")
        print(f"Tipo de dato de 'diagnosis' antes: {df['diagnosis'].dtype}")

        # Usaremos pd.get_dummies con drop_first=True para una codificación binaria eficiente.
        # Esto crea una nueva columna para una de las categorías (e.g., 'diagnosis_malignant')
        # y le asigna 1 si la observación pertenece a esa categoría, y 0 en caso contrario.
        # 'drop_first=True' elimina la redundancia: si no es 1 en 'diagnosis_malignant', entonces es 0 (benign).
        # 'dtype=int' asegura que la nueva columna sea de tipo entero (0 o 1).

        # Es importante saber qué categoría se "dropea" para interpretar el 0 y 1.
        # pd.get_dummies ordena las categorías alfabéticamente y dropea la primera.
        # Si nuestras categorías son 'benign' y 'malignant':
        # 'benign' es la primera alfabéticamente, así que se crea 'diagnosis_malignant'.
        # 'diagnosis_malignant' = 1 si la etiqueta original era 'malignant'.
        # 'diagnosis_malignant' = 0 si la etiqueta original era 'benign'.
        
        print("\nCodificando 'diagnosis' a formato numérico (0/1)...")
        df = pd.get_dummies(df, columns=['diagnosis'], drop_first=True, dtype=int)
        
        # Identificamos el nuevo nombre de la columna objetivo codificada.
        # Debería ser 'diagnosis_malignant' si 'benign' fue la primera categoría alfabética.
        new_encoded_target_column = [col for col in df.columns if col.startswith('diagnosis_')]
        
        if new_encoded_target_column:
            new_encoded_target_column_name = new_encoded_target_column[0]
            print(f"La columna 'diagnosis' ha sido reemplazada por '{new_encoded_target_column_name}'.")
            print(f"En '{new_encoded_target_column_name}': 1 representa la clase que NO fue eliminada (ej. 'malignant'), 0 representa la clase eliminada (ej. 'benign').")
            print(f"Valores únicos en '{new_encoded_target_column_name}': {df[new_encoded_target_column_name].unique().tolist()}")
        else:
            print("Advertencia: No se encontró la nueva columna objetivo después de get_dummies. Revisar nombres.")

        print("\nVisualización de las primeras filas del DataFrame con 'diagnosis' codificada:")
        display(df.head())
        print(f"\nDimensiones del DataFrame actualizadas: {df.shape}")
        print("\nInformación de las columnas actualizada:")
        df.info()
        
    elif 'diagnosis' in df.columns and np.issubdtype(df['diagnosis'].dtype, np.number):
        print("\nLa columna 'diagnosis' ya parece ser numérica. No se requiere codificación adicional en este paso.")
        if df['diagnosis'].nunique() == 2:
            print(f"Valores únicos en 'diagnosis' numérica: {df['diagnosis'].unique().tolist()}")
        else:
            print(f"Advertencia: 'diagnosis' es numérica pero tiene {df['diagnosis'].nunique()} valores únicos. Se esperaba una variable binaria.")
    else:
        print("\nLa columna 'diagnosis' no se encontró o no es de un tipo adecuado para codificar con get_dummies en este flujo.")
        print(f"Columnas actuales: {df.columns.tolist()}")

else:
    print("El DataFrame `df` está vacío o no definido. No se puede proceder con la codificación.")

# Nota: A partir de esta celda, la variable `df` contendrá la columna objetivo ya codificada numéricamente
# (por ejemplo, como 'diagnosis_malignant' con valores 0 y 1).

In [ ]:
# PREPROCESAMIENTO - DIVISIÓN EN CONJUNTOS DE ENTRENAMIENTO Y PRUEBA
# ---------------------------------------------------------------------------
# Este es un paso fundamental en cualquier proyecto de Machine Learning.
# Dividimos nuestro conjunto de datos en dos subconjuntos separados e independientes:
# 1. Conjunto de Entrenamiento (Training Set): Se utiliza para "enseñar" o "ajustar" el modelo.
#    El modelo aprenderá los patrones y relaciones a partir de estos datos.
# 2. Conjunto de Prueba (Test Set): Se utiliza para evaluar el rendimiento del modelo una vez entrenado.
#    Estos datos son "nuevos" para el modelo (no los ha visto durante el entrenamiento),
#    lo que nos da una medida más realista de cómo generalizará a datos futuros.

# from sklearn.model_selection import train_test_split # Ya importado globalmente.
# pandas y numpy también.

if 'df' in locals() and not df.empty:
    print("--- Preprocesamiento: División de Datos en Conjuntos de Entrenamiento y Prueba ---")

    # Identificar la columna objetivo codificada (debería ser algo como 'diagnosis_malignant' de la Celda 11).
    encoded_target_col_name_split = None
    potential_encoded_targets = [col for col in df.columns if 'diagnosis_' in col and df[col].nunique() == 2]
    if potential_encoded_targets:
        encoded_target_col_name_split = potential_encoded_targets[0] # Tomamos la primera que coincida
    else:
        # Intento de rescate si el nombre no es el esperado
        for col_candidate in ['target_is_malignant', 'target', 'label', 'class', 'status_encoded', 'output']:
            if col_candidate in df.columns and df[col_candidate].nunique() == 2 and pd.api.types.is_numeric_dtype(df[col_candidate]):
                encoded_target_col_name_split = col_candidate
                break
        if not encoded_target_col_name_split and df.iloc[:, -1].nunique() == 2 and pd.api.types.is_numeric_dtype(df.iloc[:, -1]):
            encoded_target_col_name_split = df.columns[-1]
            print(f"Advertencia: No se encontró un nombre esperado para el objetivo codificado, usando la última columna numérica binaria: '{encoded_target_col_name_split}'.")


    if encoded_target_col_name_split and encoded_target_col_name_split in df.columns:
        print(f"La variable objetivo para la división será: '{encoded_target_col_name_split}'")

        # Separamos las características predictoras (X) de la variable objetivo (y).
        X = df.drop(encoded_target_col_name_split, axis=1)
        y = df[encoded_target_col_name_split]

        print(f"\nDimensiones de X (características predictoras): {X.shape}")
        print(f"Dimensiones de y (variable objetivo): {y.shape}")

        # Verificamos la distribución de la variable objetivo antes de la división.
        print(f"\nDistribución de '{encoded_target_col_name_split}' en el conjunto completo (y):")
        print(y.value_counts(normalize=True) * 100)
        
        # Verificamos que todas las columnas en X sean numéricas.
        # El dataset load_breast_cancer ya tiene todas las features numéricas.
        non_numeric_features_in_X = X.select_dtypes(exclude=np.number).columns
        if not non_numeric_features_in_X.empty:
            print(f"\n¡Atención! Se encontraron columnas no numéricas en X: {non_numeric_features_in_X.tolist()}")
            print("Estas columnas deben ser codificadas o eliminadas antes de entrenar la mayoría de los modelos.")
            # Aquí se podría añadir lógica para codificar estas columnas si fuera un dataset diferente.
            # Para este proyecto, no debería ocurrir.
        else:
            print("\nConfirmado: Todas las características en X son numéricas.")


        # Realizamos la división.
        # test_size=0.25: Reservamos el 25% de los datos para el conjunto de prueba.
        # random_state=42: Semilla para la aleatoriedad, asegura que la división sea la misma cada vez que se ejecuta.
        # stratify=y: ¡Muy importante! Asegura que la proporción de clases en 'y' (0s y 1s)
        #              se mantenga aproximadamente igual tanto en el conjunto de entrenamiento como en el de prueba.
        #              Esto es crucial si hay desequilibrio de clases.
        print("\nRealizando la división en entrenamiento (75%) y prueba (25%)...")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, 
            test_size=0.25, 
            random_state=42, 
            stratify=y
        )

        print("\n¡División completada!")
        print(f"  Dimensiones de X_train (entrenamiento): {X_train.shape}")
        print(f"  Dimensiones de X_test (prueba): {X_test.shape}")
        print(f"  Dimensiones de y_train (objetivo entrenamiento): {y_train.shape}")
        print(f"  Dimensiones de y_test (objetivo prueba): {y_test.shape}")

        # Verificamos la estratificación.
        print(f"\nDistribución de '{encoded_target_col_name_split}' en y_train (entrenamiento):")
        print(y_train.value_counts(normalize=True) * 100)
        print(f"\nDistribución de '{encoded_target_col_name_split}' en y_test (prueba):")
        print(y_test.value_counts(normalize=True) * 100)
        if abs((y_train.value_counts(normalize=True) - y_test.value_counts(normalize=True)).max()) < 0.02: # Pequeña tolerancia
             print("Confirmado: Las proporciones de la variable objetivo son similares en ambos conjuntos gracias a 'stratify=y'.")
        else:
             print("Advertencia: Las proporciones del objetivo difieren un poco entre train/test a pesar de stratify. Revisar.")


        # Última revisión: la escala de las características en X_train.
        # Esto nos dará una pista para el siguiente paso (escalado).
        print("\nAnálisis preliminar de la escala de algunas características en X_train:")
        numeric_cols_in_X_train_check = X_train.select_dtypes(include=np.number).columns
        if not numeric_cols_in_X_train_check.empty:
            sample_cols_for_scale_check = np.random.choice(
                numeric_cols_in_X_train_check, 
                min(5, len(numeric_cols_in_X_train_check)), # Muestra hasta 5 columnas
                replace=False
            )
            display(X_train[sample_cols_for_scale_check].describe().T[['min', 'max', 'mean', 'std']])
            
            # Comprobación simple de rangos amplios.
            feature_ranges_X_train = X_train[numeric_cols_in_X_train_check].max() - X_train[numeric_cols_in_X_train_check].min()
            if (feature_ranges_X_train > 100).any() or (X_train[numeric_cols_in_X_train_check].std() > 50).any():
                print("\nObservación importante: Se detecta una variación significativa en las escalas (rangos/desviaciones estándar) de las características.")
                print("Es altamente recomendable aplicar escalado (estandarización o normalización) a X_train y X_test.")
            else:
                print("\nLas escalas de las características parecen estar en un rango razonable, aunque el escalado suele ser beneficioso.")
        else:
            print("\nNo se encontraron características numéricas en X_train para analizar su escala.")
    else:
        print("\nError: No se pudo identificar la columna objetivo codificada para realizar la división.")
        print(f"Columnas disponibles en el DataFrame: {df.columns.tolist()}")
else:
    print("El DataFrame `df` está vacío o no definido. No se puede proceder con la división de datos.")

In [ ]:
# PREPROCESAMIENTO - ESCALADO DE CARACTERÍSTICAS (ESTANDARIZACIÓN)
# -----------------------------------------------------------------------
# Como se observó en la celda anterior, las características numéricas tienen diferentes rangos.
# El escalado es importante para modelos sensibles a la magnitud de las características (e.g., SVM, Regresión Logística, KNN).
# Usaremos StandardScaler para estandarizar las características (media 0, desviación estándar 1).

from sklearn.preprocessing import StandardScaler
import pandas as pd # Asegurarse de que pandas esté disponible

if 'X_train' in locals() and 'X_test' in locals():
    print("--- Escalado de Características (Estandarización) ---")

    # Verificar que X_train y X_test no están vacíos y son DataFrames
    if not X_train.empty and not X_test.empty:
        
        # Crear el objeto StandardScaler
        scaler = StandardScaler()

        # Obtener los nombres de las columnas de X_train (asumiendo que son todas numéricas en este punto)
        # Si hubo codificación OHE en la celda 12 y se mezclaron con numéricas, esto debería funcionar.
        # Dado el output de la celda 12, X_train ya es completamente numérico.
        
        X_train_columns = X_train.columns
        X_test_columns = X_test.columns

        print(f"\nColumnas en X_train antes del escalado: {X_train.shape[1]}")
        print(f"Columnas en X_test antes del escalado: {X_test.shape[1]}")

        # Ajustar el scaler CON SOLO DATOS DE ENTRENAMIENTO y luego transformar ambos conjuntos
        print("\nAjustando StandardScaler con X_train y transformando X_train y X_test...")
        X_train_scaled_np = scaler.fit_transform(X_train)
        X_test_scaled_np = scaler.transform(X_test)

        # Convertir los arrays de NumPy de vuelta a DataFrames de Pandas
        # Es importante mantener los nombres de las columnas y los índices.
        X_train_scaled = pd.DataFrame(X_train_scaled_np, columns=X_train_columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(X_test_scaled_np, columns=X_test_columns, index=X_test.index)
        
        print("\nEscalado completado.")
        
        print("\n--- Estadísticas Descriptivas de X_train_scaled (primeras 5 columnas) ---")
        # Usar display si estás en un entorno Jupyter/IPython para mejor formato
        # De lo contrario, print(X_train_scaled.iloc[:, :5].describe().T[['min', 'max', 'mean', 'std']])
        try:
            display(X_train_scaled.iloc[:, :5].describe().T[['min', 'max', 'mean', 'std']])
        except NameError: # Si display no está definido (ej. script puro de Python)
            print(X_train_scaled.iloc[:, :5].describe().T[['min', 'max', 'mean', 'std']])
        
        print("\n--- Estadísticas Descriptivas de X_test_scaled (primeras 5 columnas) ---")
        try:
            display(X_test_scaled.iloc[:, :5].describe().T[['min', 'max', 'mean', 'std']])
        except NameError:
            print(X_test_scaled.iloc[:, :5].describe().T[['min', 'max', 'mean', 'std']])
        
        print("\nLas medias deberían ser cercanas a 0 y las desviaciones estándar cercanas a 1 para X_train_scaled.")
        print("X_test_scaled se transforma usando el scaler ajustado en X_train, por lo que sus medias/std pueden variar un poco más.")

        # Guardar los dataframes escalados para las siguientes celdas
        # Sobrescribimos X_train y X_test con sus versiones escaladas para simplificar las celdas de modelado.
        # Si se quisiera comparar modelos con y sin escalado, se deberían usar nombres diferentes.
        X_train = X_train_scaled
        X_test = X_test_scaled
        
        print("\nVariables X_train y X_test actualizadas con los datos escalados.")
        
    else:
        print("X_train o X_test están vacíos. No se puede realizar el escalado.")
else:
    print("Las variables X_train y X_test no están definidas. Ejecuta la celda 12 primero.")

In [ ]:
# ENTRENAMIENTO Y EVALUACIÓN - REGRESIÓN LOGÍSTICA
# ---------------------------------------------------------
# Ahora que los datos están preprocesados (divididos y escalados),
# podemos entrenar nuestro primer modelo. Empezaremos con Regresión Logística.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

print("--- Entrenamiento y Evaluación: Regresión Logística ---")

# Verificar si las variables necesarias existen
if 'X_train' in locals() and 'y_train' in locals() and 'X_test' in locals() and 'y_test' in locals():

    # 1. Inicializar y entrenar el modelo
    # 'liblinear' es un buen solver para datasets pequeños y problemas binarios.
    # random_state para reproducibilidad.
    log_reg = LogisticRegression(random_state=42, solver='liblinear')
    log_reg.fit(X_train, y_train) # X_train ya está escalado de la celda anterior

    print("\nModelo Regresión Logística entrenado.")

    # 2. Realizar predicciones en el conjunto de prueba
    y_pred_log_reg = log_reg.predict(X_test) # X_test ya está escalado
    y_proba_log_reg = log_reg.predict_proba(X_test)[:, 1] # Probabilidades para la clase positiva (Malignant)

    # 3. Evaluar el modelo
    print(f"\nAccuracy en el conjunto de prueba: {accuracy_score(y_test, y_pred_log_reg):.4f}")

    print("\nClassification Report:")
    # Asumiendo que 0 es Benign (False) y 1 es Malignant (True) después de la codificación de y
    # Esto se basa en cómo 'diagnosis_malignant' fue probablemente creado (True para malignant)
    # y cómo LabelEncoder usualmente asigna 0 a la primera etiqueta alfabética y 1 a la segunda si no se especifica.
    # Si 'diagnosis_malignant' es directamente True/False, esto funciona.
    # Si usaste LabelEncoder explícitamente en 'y', puedes usar le.classes_
    # En tu caso, `y` es directamente la columna `diagnosis_malignant` que es True/False,
    # y train_test_split mantiene estos valores.
    target_names_report = ['Benign (False)', 'Malignant (True)']
    print(classification_report(y_test, y_pred_log_reg, target_names=target_names_report))

    print("\nConfusion Matrix:")
    cm_log_reg = confusion_matrix(y_test, y_pred_log_reg)
    print(cm_log_reg)

    # Visualizar la Matriz de Confusión
    plt.figure(figsize=(8,6))
    # Los display_labels deben coincidir con el orden de las clases en la matriz de confusión (0, 1)
    disp_log_reg = ConfusionMatrixDisplay(confusion_matrix=cm_log_reg, display_labels=target_names_report)
    disp_log_reg.plot(cmap=plt.cm.Blues, values_format='d')
    plt.title('Confusion Matrix - Logistic Regression')
    plt.show()

    # Calcular y mostrar ROC AUC Score
    roc_auc_log_reg = roc_auc_score(y_test, y_proba_log_reg)
    print(f"\nROC AUC Score: {roc_auc_log_reg:.4f}")

    # Dibujar la Curva ROC
    fpr, tpr, thresholds = roc_curve(y_test, y_proba_log_reg)
    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc_log_reg:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--') # Línea de referencia (azar)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity/Recall)')
    plt.title('Receiver Operating Characteristic (ROC) - Logistic Regression')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.show()

else:
    print("Asegúrate de que X_train, y_train, X_test, y y_test estén definidos y sean correctos.")
    print("Ejecuta las celdas de preprocesamiento (especialmente división y escalado) primero.")

In [ ]:
# ENTRENAMIENTO Y EVALUACIÓN - RANDOM FOREST CLASSIFIER
# ------------------------------------------------------------
# A continuación, probaremos con un modelo basado en ensambles: Random Forest.
# Estos modelos suelen ser más robustos y potentes que los modelos lineales simples.

from sklearn.ensemble import RandomForestClassifier
# Las métricas y matplotlib.pyplot ya deberían estar importados de la celda anterior,
# pero es buena práctica asegurarse o reimportarlos si es una nueva sesión/celda aislada.
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt

print("--- Entrenamiento y Evaluación: Random Forest Classifier ---")

# Verificar si las variables necesarias existen
if 'X_train' in locals() and 'y_train' in locals() and 'X_test' in locals() and 'y_test' in locals():

    # 1. Inicializar y entrenar el modelo
    # n_estimators: número de árboles en el bosque.
    # random_state para reproducibilidad.
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced_subsample')
    rf_clf.fit(X_train, y_train) # X_train ya está escalado

    print("\nModelo Random Forest Classifier entrenado.")

    # 2. Realizar predicciones en el conjunto de prueba
    y_pred_rf = rf_clf.predict(X_test) # X_test ya está escalado
    y_proba_rf = rf_clf.predict_proba(X_test)[:, 1] # Probabilidades para la clase positiva

    # 3. Evaluar el modelo
    print(f"\nAccuracy en el conjunto de prueba: {accuracy_score(y_test, y_pred_rf):.4f}")

    print("\nClassification Report:")
    target_names_report = ['Benign (False)', 'Malignant (True)'] # Mismos que en LogReg
    print(classification_report(y_test, y_pred_rf, target_names=target_names_report))

    print("\nConfusion Matrix:")
    cm_rf = confusion_matrix(y_test, y_pred_rf)
    print(cm_rf)

    # Visualizar la Matriz de Confusión
    plt.figure(figsize=(8,6))
    disp_rf = ConfusionMatrixDisplay(confusion_matrix=cm_rf, display_labels=target_names_report)
    disp_rf.plot(cmap=plt.cm.Greens, values_format='d') # Diferente color para distinguir
    plt.title('Confusion Matrix - Random Forest Classifier')
    plt.show()

    # Calcular y mostrar ROC AUC Score
    roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
    print(f"\nROC AUC Score: {roc_auc_rf:.4f}")

    # Dibujar la Curva ROC
    fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_proba_rf)
    plt.figure(figsize=(8,6))
    plt.plot(fpr_rf, tpr_rf, color='green', lw=2, label=f'ROC curve (area = {roc_auc_rf:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity/Recall)')
    plt.title('Receiver Operating Characteristic (ROC) - Random Forest')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.show()

else:
    print("Asegúrate de que X_train, y_train, X_test, y y_test estén definidos y sean correctos.")
    print("Ejecuta las celdas de preprocesamiento (especialmente división y escalado) primero.")

In [ ]:
# ENTRENAMIENTO Y EVALUACIÓN - SUPPORT VECTOR MACHINE (SVC)
# -----------------------------------------------------------------
# Continuamos explorando modelos, ahora con Support Vector Machines (SVM),
# específicamente Support Vector Classifier (SVC) para clasificación.
# Los SVM pueden ser muy efectivos, especialmente en espacios de alta dimensión.

from sklearn.svm import SVC
# Las métricas y matplotlib.pyplot ya deberían estar importados.
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt

print("--- Entrenamiento y Evaluación: Support Vector Machine (SVC) ---")

# Verificar si las variables necesarias existen
if 'X_train' in locals() and 'y_train' in locals() and 'X_test' in locals() and 'y_test' in locals():

    # 1. Inicializar y entrenar el modelo
    # probability=True es necesario para poder usar predict_proba y calcular ROC AUC.
    # Puede hacer que el entrenamiento sea un poco más lento.
    # random_state para reproducibilidad si el SVC tiene algún componente estocástico (menos común).
    # C es el parámetro de regularización. Un valor más alto significa menos regularización.
    # kernel='rbf' (Radial Basis Function) es una elección común y potente.
    svc_clf = SVC(probability=True, random_state=42, C=1.0, kernel='rbf', class_weight='balanced')
    svc_clf.fit(X_train, y_train) # X_train ya está escalado

    print("\nModelo Support Vector Machine (SVC) entrenado.")

    # 2. Realizar predicciones en el conjunto de prueba
    y_pred_svc = svc_clf.predict(X_test) # X_test ya está escalado
    y_proba_svc = svc_clf.predict_proba(X_test)[:, 1] # Probabilidades para la clase positiva

    # 3. Evaluar el modelo
    print(f"\nAccuracy en el conjunto de prueba: {accuracy_score(y_test, y_pred_svc):.4f}")

    print("\nClassification Report:")
    target_names_report = ['Benign (False)', 'Malignant (True)'] # Mismos que antes
    print(classification_report(y_test, y_pred_svc, target_names=target_names_report))

    print("\nConfusion Matrix:")
    cm_svc = confusion_matrix(y_test, y_pred_svc)
    print(cm_svc)

    # Visualizar la Matriz de Confusión
    plt.figure(figsize=(8,6))
    disp_svc = ConfusionMatrixDisplay(confusion_matrix=cm_svc, display_labels=target_names_report)
    disp_svc.plot(cmap=plt.cm.Oranges, values_format='d') # Diferente color
    plt.title('Confusion Matrix - Support Vector Machine (SVC)')
    plt.show()

    # Calcular y mostrar ROC AUC Score
    roc_auc_svc = roc_auc_score(y_test, y_proba_svc)
    print(f"\nROC AUC Score: {roc_auc_svc:.4f}")

    # Dibujar la Curva ROC
    fpr_svc, tpr_svc, thresholds_svc = roc_curve(y_test, y_proba_svc)
    plt.figure(figsize=(8,6))
    plt.plot(fpr_svc, tpr_svc, color='orange', lw=2, label=f'ROC curve (area = {roc_auc_svc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity/Recall)')
    plt.title('Receiver Operating Characteristic (ROC) - SVC')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.show()

else:
    print("Asegúrate de que X_train, y_train, X_test, y y_test estén definidos y sean correctos.")
    print("Ejecuta las celdas de preprocesamiento (especialmente división y escalado) primero.")

In [ ]:
# ENTRENAMIENTO Y EVALUACIÓN - XGBOOST CLASSIFIER
# -------------------------------------------------------
# Finalmente, probaremos con XGBoost, un algoritmo de Gradient Boosting
# muy popular y eficiente, conocido por su alto rendimiento.

import xgboost as xgb
# Las métricas y matplotlib.pyplot ya deberían estar importados.
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt

print("--- Entrenamiento y Evaluación: XGBoost Classifier ---")

# Verificar si las variables necesarias existen
if 'X_train' in locals() and 'y_train' in locals() and 'X_test' in locals() and 'y_test' in locals():

    # 1. Inicializar y entrenar el modelo
    # random_state para reproducibilidad.
    # use_label_encoder=False para evitar una advertencia común con versiones recientes de XGBoost.
    # eval_metric='logloss' es una métrica de evaluación común para clasificación binaria durante el entrenamiento (si se usa early stopping).
    # n_estimators: número de árboles (rondas de boosting).
    # learning_rate: reduce la contribución de cada árbol.
    # max_depth: profundidad máxima de cada árbol.
    # scale_pos_weight: útil para clases desbalanceadas. Se calcula como (count(negative_class) / count(positive_class)).
    # Contemos las clases en y_train para scale_pos_weight
    count_benign_train = (y_train == 0).sum() # Asumiendo 0 para benigno (False)
    count_malignant_train = (y_train == 1).sum() # Asumiendo 1 para maligno (True)
    
    scale_pos_weight_xgb = 1 # Valor por defecto si algo falla
    if count_malignant_train > 0: # Evitar división por cero
        scale_pos_weight_xgb = count_benign_train / count_malignant_train
    print(f"Calculado scale_pos_weight para XGBoost: {scale_pos_weight_xgb:.2f}")

    xgb_clf = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3, # Profundidad típica para empezar
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss',
        scale_pos_weight=scale_pos_weight_xgb # Para manejar desequilibrio
    )
    xgb_clf.fit(X_train, y_train) # X_train ya está escalado

    print("\nModelo XGBoost Classifier entrenado.")

    # 2. Realizar predicciones en el conjunto de prueba
    y_pred_xgb = xgb_clf.predict(X_test) # X_test ya está escalado
    y_proba_xgb = xgb_clf.predict_proba(X_test)[:, 1] # Probabilidades para la clase positiva

    # 3. Evaluar el modelo
    print(f"\nAccuracy en el conjunto de prueba: {accuracy_score(y_test, y_pred_xgb):.4f}")

    print("\nClassification Report:")
    target_names_report = ['Benign (False)', 'Malignant (True)'] # Mismos que antes
    print(classification_report(y_test, y_pred_xgb, target_names=target_names_report))

    print("\nConfusion Matrix:")
    cm_xgb = confusion_matrix(y_test, y_pred_xgb)
    print(cm_xgb)

    # Visualizar la Matriz de Confusión
    plt.figure(figsize=(8,6))
    disp_xgb = ConfusionMatrixDisplay(confusion_matrix=cm_xgb, display_labels=target_names_report)
    disp_xgb.plot(cmap=plt.cm.Purples, values_format='d') # Diferente color
    plt.title('Confusion Matrix - XGBoost Classifier')
    plt.show()

    # Calcular y mostrar ROC AUC Score
    roc_auc_xgb = roc_auc_score(y_test, y_proba_xgb)
    print(f"\nROC AUC Score: {roc_auc_xgb:.4f}")

    # Dibujar la Curva ROC
    fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y_test, y_proba_xgb)
    plt.figure(figsize=(8,6))
    plt.plot(fpr_xgb, tpr_xgb, color='purple', lw=2, label=f'ROC curve (area = {roc_auc_xgb:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity/Recall)')
    plt.title('Receiver Operating Characteristic (ROC) - XGBoost')
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.show()

else:
    print("Asegúrate de que X_train, y_train, X_test, y y_test estén definidos y sean correctos.")
    print("Ejecuta las celdas de preprocesamiento (especialmente división y escalado) primero.")

In [ ]:
# COMPARACIÓN DE MODELOS
# ------------------------------
# Ahora que hemos entrenado y evaluado varios modelos,
# vamos a comparar sus métricas clave para determinar cuál tuvo el mejor rendimiento.

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # Para np.nan si alguna variable no estuviera definida

print("--- Comparación de Modelos ---")

# Asegurarse de que todas las variables de predicción y ROC AUC estén disponibles.
# Usaremos try-except para manejar el caso de que alguna no se haya ejecutado.

model_data = []

try:
    model_data.append({
        'Model': 'Logistic Regression',
        'Accuracy': accuracy_score(y_test, y_pred_log_reg),
        'ROC AUC Score': roc_auc_log_reg,
        'FN': confusion_matrix(y_test, y_pred_log_reg)[1, 0], # Fila 1, Columna 0 para FN
        'FP': confusion_matrix(y_test, y_pred_log_reg)[0, 1]  # Fila 0, Columna 1 para FP
    })
except NameError:
    print("Advertencia: Resultados de Regresión Logística no encontrados.")
    model_data.append({'Model': 'Logistic Regression', 'Accuracy': np.nan, 'ROC AUC Score': np.nan, 'FN': np.nan, 'FP': np.nan})

try:
    model_data.append({
        'Model': 'Random Forest',
        'Accuracy': accuracy_score(y_test, y_pred_rf),
        'ROC AUC Score': roc_auc_rf,
        'FN': confusion_matrix(y_test, y_pred_rf)[1, 0],
        'FP': confusion_matrix(y_test, y_pred_rf)[0, 1]
    })
except NameError:
    print("Advertencia: Resultados de Random Forest no encontrados.")
    model_data.append({'Model': 'Random Forest', 'Accuracy': np.nan, 'ROC AUC Score': np.nan, 'FN': np.nan, 'FP': np.nan})

try:
    model_data.append({
        'Model': 'SVC',
        'Accuracy': accuracy_score(y_test, y_pred_svc),
        'ROC AUC Score': roc_auc_svc,
        'FN': confusion_matrix(y_test, y_pred_svc)[1, 0],
        'FP': confusion_matrix(y_test, y_pred_svc)[0, 1]
    })
except NameError:
    print("Advertencia: Resultados de SVC no encontrados.")
    model_data.append({'Model': 'SVC', 'Accuracy': np.nan, 'ROC AUC Score': np.nan, 'FN': np.nan, 'FP': np.nan})

try:
    model_data.append({
        'Model': 'XGBoost',
        'Accuracy': accuracy_score(y_test, y_pred_xgb),
        'ROC AUC Score': roc_auc_xgb,
        'FN': confusion_matrix(y_test, y_pred_xgb)[1, 0],
        'FP': confusion_matrix(y_test, y_pred_xgb)[0, 1]
    })
except NameError:
    print("Advertencia: Resultados de XGBoost no encontrados.")
    model_data.append({'Model': 'XGBoost', 'Accuracy': np.nan, 'ROC AUC Score': np.nan, 'FN': np.nan, 'FP': np.nan})


results_df = pd.DataFrame(model_data)
# Ordenar por FN (ascendente, menos es mejor) y luego por ROC AUC (descendente) como desempate
results_df = results_df.sort_values(by=['FN', 'ROC AUC Score'], ascending=[True, False]).reset_index(drop=True)

print("\nResultados de los Modelos (Ordenados por FN y luego ROC AUC):")
# Usar display si estás en un entorno Jupyter/IPython para mejor formato
try:
    display(results_df)
except NameError:
    print(results_df)

# --- Visualizaciones de Comparación ---

plt.style.use('seaborn-v0_8-whitegrid') # Asegurar estilo consistente

# Comparación de Accuracy
plt.figure(figsize=(10, 6))
ax_acc = sns.barplot(x='Accuracy', y='Model', data=results_df.sort_values(by='Accuracy', ascending=False), palette='viridis', orient='h')
plt.title('Comparación de Accuracy de Modelos', fontsize=16)
plt.xlabel('Accuracy', fontsize=14)
plt.ylabel('Modelo', fontsize=14)
plt.xlim(0.90, 1.005) # Ajustar límites para mejor visualización
for i in ax_acc.containers:
    ax_acc.bar_label(i, fmt='%.4f', padding=3)
plt.tight_layout()
plt.show()

# Comparación de ROC AUC Score
plt.figure(figsize=(10, 6))
ax_roc = sns.barplot(x='ROC AUC Score', y='Model', data=results_df.sort_values(by='ROC AUC Score', ascending=False), palette='magma', orient='h')
plt.title('Comparación de ROC AUC Score de Modelos', fontsize=16)
plt.xlabel('ROC AUC Score', fontsize=14)
plt.ylabel('Modelo', fontsize=14)
plt.xlim(0.98, 1.005) # Ajustar límites
for i in ax_roc.containers:
    ax_roc.bar_label(i, fmt='%.4f', padding=3)
plt.tight_layout()
plt.show()

# Comparación de Falsos Negativos (FN)
plt.figure(figsize=(10, 6))
ax_fn = sns.barplot(x='FN', y='Model', data=results_df.sort_values(by='FN', ascending=True), palette='coolwarm', orient='h')
plt.title('Comparación de Falsos Negativos (FN) de Modelos', fontsize=16)
plt.xlabel('Número de Falsos Negativos', fontsize=14)
plt.ylabel('Modelo', fontsize=14)
# Ajustar límites, puede ser de 0 a un poco más del máximo FN
max_fn = results_df['FN'].max()
plt.xlim(-0.5, max_fn + 1 if not pd.isna(max_fn) else 5)
for i in ax_fn.containers:
    ax_fn.bar_label(i, fmt='%d', padding=3) # Formato entero para FN
plt.tight_layout()
plt.show()

# Comparación de Falsos Positivos (FP)
plt.figure(figsize=(10, 6))
ax_fp = sns.barplot(x='FP', y='Model', data=results_df.sort_values(by='FP', ascending=True), palette='Spectral', orient='h')
plt.title('Comparación de Falsos Positivos (FP) de Modelos', fontsize=16)
plt.xlabel('Número de Falsos Positivos', fontsize=14)
plt.ylabel('Modelo', fontsize=14)
max_fp = results_df['FP'].max()
plt.xlim(-0.5, max_fp + 1 if not pd.isna(max_fp) else 2)
for i in ax_fp.containers:
    ax_fp.bar_label(i, fmt='%d', padding=3) # Formato entero para FP
plt.tight_layout()
plt.show()


# --- Conclusión Preliminar ---
print("\n--- Conclusión Preliminar ---")
if not results_df.empty and not results_df['FN'].isnull().all():
    best_overall_model = results_df.iloc[0] # El primero después de ordenar
    print(f"Considerando la prioridad de minimizar Falsos Negativos (FN) y luego maximizar ROC AUC:")
    print(f"El modelo más prometedor es **{best_overall_model['Model']}** con:")
    print(f"  - Falsos Negativos (FN): {best_overall_model['FN']:.0f}")
    print(f"  - Falsos Positivos (FP): {best_overall_model['FP']:.0f}")
    print(f"  - Accuracy: {best_overall_model['Accuracy']:.4f}")
    print(f"  - ROC AUC Score: {best_overall_model['ROC AUC Score']:.4f}")

    print("\nTodos los modelos han mostrado un rendimiento general muy alto en este dataset.")
    print("La elección final del 'mejor' modelo a menudo depende del costo relativo de los diferentes tipos de errores en el contexto específico del problema.")
    print("Para el diagnóstico de cáncer, minimizar los Falsos Negativos (no detectar un caso maligno) suele ser la máxima prioridad.")
else:
    print("No se pudieron recopilar suficientes datos para una conclusión.")